# Assumptions for the data rate estimates per observing modes

## Imaging

In [1]:
#Image size assuming observing parameters, e.g. baseline lengths, sampling, frequency
station_diameter=39
max_baseline_length=70e3
n_pols=4
n_chan=1000
#The 7.5 factor comes from assuming 2xFOV, 3 times sampling of the PSF, and the 1.25 factor in lambda/D
image_size=7.5*(max_baseline_length/station_diameter)**2 * (32/8)
print(image_size)
final_cube= n_pols * n_chan * image_size
print(f'Image size in MBs = {round(final_cube/1e6,5)}')
print(f'Image size in GBs = {round(final_cube/1e9,5)}')
print(f'Image size in TBs = {round(final_cube/1e12,5)}')
print(f'Image size in PBs = {round(final_cube/1e15,5)}')

96646942.80078895
Image size in MBs = 386587.7712
Image size in GBs = 386.58777
Image size in TBs = 0.38659
Image size in PBs = 0.00039


In [ ]:
#Assuming 
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
def image_size_low(station_diameter,max_baseline_length,n_chan_image,n_pols,n_products,number_beams):
    image_size=7.5*(max_baseline_length/station_diameter)**2 * (32/8)
    #Assuming 4 imaging products, image, model, residual, psf (1 channel only)
    final_cube= number_beams * n_pols * n_chan_image * image_size *(n_products-1) + (number_beams*image_size) #last part is the PSF which will be 1 channel
    sGB=(1024*1024*1024)
    sTB=(1024*1024*1024*1024)
    sPB=(1024*1024*1024*1024*1024)
    print(f'cube size {round(final_cube/sGB,4)}GB, {round(final_cube/sTB,10)}TB,{round(final_cube/sPB,6)}PB')
    totalTB=final_cube/sTB
    return totalTB

# Visibility Data
### Definition from LOW which is based on the ms format from Lofar (from Sarrvesh)

In [ ]:
#Visbility data for LOW which is based on the ms format from Lofar (from Sarrvesh)
ant_number=64
n_baselines=(ant_number*(ant_number-1))/2
#n_baselines=1000
obs_t=600
int_time=15
n_chan=5000
n_rows = n_baselines * int( obs_t / int_time)
# What does a single row in an averaged MS contain?
sb_size = n_rows * ((7*8) + \
                         (4+(4*n_chan)) + \
                         (4*11) + \
                         (8*1) + \
                         (4) + \
                         (4 * (8 + 8*n_chan + 4*n_chan)))
# Convert byte length to GB
sb_size /= (1024*1024*1024)

print(round(sb_size,3),'GB')

In [ ]:
#Current definition used to estimate visibilty data size
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
default_time_res=0.85 #sesconds
def visibility_size_low(number_stations,n_chan,obs_time,int_time,freq_res):
    n_baselines=(number_stations*(number_stations+1))/2
    n_rows = n_baselines * int( obs_time / int_time)
    default_frequency=5.4e3 #default frequency resolution =5.4kHz
    frequency_scaling=freq_res/default_frequency
    n_chan=n_chan/(frequency_scaling)
    #4 polarizations already taken into account.
    sb_size = n_rows * ((7*8) + (4+(4*n_chan)) + (4*11) + (8*1) + (4) + (4 * (8 + 8*n_chan + 4*n_chan)))
    sGB=(1024*1024*1024)
    sTB=(1024*1024*1024*1024)
    sPB=(1024*1024*1024*1024*1024)
    print(f'visib size {round(sb_size/sGB,3)}GB,{round(sb_size/sTB,4)}TB,{round(sb_size/sPB,4)}PB')
    return sb_size/sTB

## Non imaging products (PSS, PST, Dynamic spectrum, Flowthrough and Transient Buffer

In [12]:
#Assumptions for PSS, DYNAMIC, PST, FLowthrough, and Transient Buffer 
gbps_to_PB=(1/(8000*1e3))
gbps_to_TB=(1/(8000))
GBtoPB=1e6
number_beams=1
number_stations=199
#Assuming a 4 hour observing block
obs_time=4*3600
#---------------------------Flowthrough-----------------------------------------
datarate=1.92 #Gb/s
flowthrough_TB=number_beams*datarate*obs_time*gbps_to_TB
print('Flowthrough=',flowthrough_TB,'TB')

#-------------------------Dynamic spectrum----------------------------------------
datarate=2.048  #Gb/s
dynamic_TB=number_beams*datarate*obs_time*gbps_to_TB
print('Dynamic spectrum=',dynamic_TB,'TB')

#---------------------------- PST ----------------------------------------------------
datarate=2*1.33 
PST=number_beams*datarate*obs_time*gbps_to_TB
print('PST=',PST,'TB')

#---------------------------- PSS ----------------------------------------------------
#!!!!!!!!! I have assumed 5TB for PSS this needs to be confirmed !!!!!!
scan_size=10 #GB
n_beams=250 #number of pss search beams
pss_TB=8*(n_beams*(scan_size/1e3))
print('PSS=',pss_TB,'TB')

#assuming folder pulse rate 
fp=0.0839 #Gbps
#assuming single pulse is 
sp=0.0209 #Gbps
#8 is assumining max chunk of 30 minutes
pss_TB_updated=16*n_beams*fp*gbps_to_TB*100
print('PSS_updated=',pss_TB_updated,'TB')

#---------------------------Transient Buffer-----------------------------------------

buffersize_gbps=(192 * 2 * number_stations * (1/1e-6))/1e9 #bps
#assume 2 minutes of data is dumped per hour for a 4 hour block
number_hours=4
buffersize_4hrs=buffersize_gbps*gbps_to_TB*2*60*number_hours*number_beams

print('Transient Buffer=',buffersize_4hrs,'TB')

Flowthrough= 3.456 TB
Dynamic spectrum= 3.6864000000000003 TB
PST= 4.788 TB
PSS= 20.0 TB
PSS_updated= 4.195 TB
Transient Buffer= 4.58496 TB


In [18]:
#Quick estimate of data rates
datasize=80 #TB
number_of_hours=4
no_seconds=number_of_hours*3600
gbps_to_TB=(1/(8000))
datarate=datasize/no_seconds/gbps_to_TB
#frequency resolution
print('Expected data rate',round(datarate,3),'gbps')

Expected data rate 44.444 gbps
